In [1]:
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.tree import DecisionTree
from pyspark import SparkConf, SparkContext

In [2]:
from numpy import array

In [3]:
conf = SparkConf().setMaster("local").setAppName('DecisionTree')
sc = SparkContext(conf=conf)

In [4]:
def binary(YN):
    if (YN == 'Y'):
        return 1
    else:
        return 0

def mapEducation(degree):
    if (degree == 'BS'):
        return 1
    elif (degree =='MS'):
        return 2
    elif (degree == 'PhD'):
        return 3
    else:
        return 0

In [16]:
def createEndpoints(fields):
    yearsExperience = int(fields[0])
    employed = binary(fields[1])
    previousEmployers = int(fields[2])
    educationLevel = mapEducation(fields[3])
    topTier = binary(fields[4])
    interned = binary(fields[5])
    hired = binary(fields[6])

    return LabeledPoint(hired, array([yearsExperience, employed,
        previousEmployers, educationLevel, topTier, interned]))


In [17]:
rawData = sc.textFile("PastHires.csv")
header = rawData.first()
rawData = rawData.filter(lambda x: x != header) #removing the header

In [18]:
csvData = rawData.map( lambda x: x.split(","))

In [19]:
trainingData = csvData.map(createEndpoints)

In [20]:
testCandidates = [ array([10, 1, 3, 1, 0, 0])]
testData = sc.parallelize(testCandidates)

In [21]:
model = DecisionTree.trainClassifier(trainingData, numClasses=2,
                                     categoricalFeaturesInfo={1:2, 3:4, 4:2, 5:2},
                                     impurity='gini', maxDepth=5, maxBins=32)

In [22]:
predictions = model.predict(testData)

In [23]:
print("Are you hired? \n")
results = predictions.collect()
for result in results:
    print(result)

Are you hired? 

1.0
